# Introduction
The COVID-19 pandemic impacted many aspects of our lives - in particular, how mobile we are.

This analysis aims to look at two main parts:
1. How did introduction of variants affect human traffic
2. How did lockdowns affect human effect

The focus will be on 6 main cities:
1. Brisbane (Greater metro area)
2. Sydney (Greater metro area)
3. Melbourne (Greater metro area)
4. Singapore
5. Tokyo (Greater metro area)
6. New York City (5 Boroughs)

## Datasets
Google collects location data, aggregated and anonymised, for the purposes of research.

These datasets show how visits and length of stay at different places change compared to a baseline. We calculate these changes using the same kind of aggregated and anonymized data used to show popular times for places in Google Maps.

Changes for each day are compared to a baseline value for that day of the week. The baseline is the median value, for the corresponding day of the week, during the 5-week period Jan 3–Feb 6, 2020.

Google LLC "Google COVID-19 Community Mobility Reports".
https://www.google.com/covid19/mobility/ Accessed: 25 January 2022.

# Exploratory Data Analysis
First let's do a bit of exploratory data analysis - nothing too fancy.

Essentially I want to know a few things:
1. Min and max dates of the dataset
2. Min and max of each category
3. Any missing values

In [6]:
import pandas as pd
import seaborn as sns

# Format percentages correctly
pd.options.display.float_format = '{:.2%}'.format

df_australia_2020 = pd.read_csv('../input/gmr-extract-au-sg-jp/2020_AU_Region_Mobility_Report.csv')
df_australia_2021 = pd.read_csv('../input/gmr-extract-au-sg-jp/2021_AU_Region_Mobility_Report.csv')
df_australia_2022 = pd.read_csv('../input/gmr-extract-au-sg-jp/2022_AU_Region_Mobility_Report.csv')

df_singapore_2020 = pd.read_csv('../input/gmr-extract-au-sg-jp/2020_SG_Region_Mobility_Report.csv')
df_singapore_2021 = pd.read_csv('../input/gmr-extract-au-sg-jp/2021_SG_Region_Mobility_Report.csv')
df_singapore_2022 = pd.read_csv('../input/gmr-extract-au-sg-jp/2022_SG_Region_Mobility_Report.csv')

df_japan_2020 = pd.read_csv('../input/gmr-extract-au-sg-jp/2020_JP_Region_Mobility_Report.csv')
df_japan_2021 = pd.read_csv('../input/gmr-extract-au-sg-jp/2021_JP_Region_Mobility_Report.csv')
df_japan_2022 = pd.read_csv('../input/gmr-extract-au-sg-jp/2022_JP_Region_Mobility_Report.csv')

df_us_2020 = pd.read_csv('../input/gmr-extract-au-sg-jp/2020_US_Region_Mobility_Report.csv')
df_us_2021 = pd.read_csv('../input/gmr-extract-au-sg-jp/2021_US_Region_Mobility_Report.csv')
df_us_2022 = pd.read_csv('../input/gmr-extract-au-sg-jp/2022_US_Region_Mobility_Report.csv')

# Union the years together
df_australia = pd.concat([df_australia_2020, df_australia_2021, df_australia_2022])
df_singapore = pd.concat([df_singapore_2020, df_singapore_2021, df_singapore_2022])
df_japan = pd.concat([df_japan_2020, df_japan_2021, df_japan_2022])
df_us = pd.concat([df_us_2020, df_us_2021, df_us_2022])

In [7]:
def transform_formats(df):
    """Small function to clean up formatting"""
    df['date'] = pd.to_datetime(df['date'])        
    
    return df

df_australia = transform_formats(df_australia)
df_japan = transform_formats(df_japan)
df_singapore = transform_formats(df_singapore)
df_us = transform_formats(df_us)

In [8]:
def get_missing_data_points(
    df, 
    fields=[
        'date', 
        'retail_and_recreation_percent_change_from_baseline',
        'grocery_and_pharmacy_percent_change_from_baseline',
        'parks_percent_change_from_baseline',
        'transit_stations_percent_change_from_baseline',
        'workplaces_percent_change_from_baseline',
        'residential_percent_change_from_baseline'
    ]
) -> pd.DataFrame:
    """Returns the missing data points for specified fields.
    :param df
    :param type: DataFrame
    :param fields Optional defaults to specified list
    :param type: Optional[List[str]]
    """
    
    return df[df[fields].isna().any(axis=1)]

In [9]:
def describe_summary(df):
    """Statistical summary of DataFrame.
    Divides percentages by hundred for display purposes
    """
        
#     for label in [
#         'retail_and_recreation_percent_change_from_baseline',
#         'grocery_and_pharmacy_percent_change_from_baseline',
#         'parks_percent_change_from_baseline',
#         'transit_stations_percent_change_from_baseline',
#         'workplaces_percent_change_from_baseline',
#         'residential_percent_change_from_baseline'
#     ]:
#         df[label] = df[label]/100 # Convert to proper %
        
    result = df.describe(
        include='all', 
        datetime_is_numeric=True
    ).transpose()
    
    return result

In [10]:
describe_summary(df_australia)

In [11]:
get_missing_data_points(df_australia)

In [12]:
describe_summary(df_singapore)

In [13]:
get_missing_data_points(df_singapore)

In [14]:
describe_summary(df_japan)

In [15]:
get_missing_data_points(df_japan)

In [16]:
describe_summary(df_us)

In [17]:
get_missing_data_points(df_us)

Therefore the answers to my questions are:
1. The datasets range from 15th February 2020 to 21 January 2022 (as I accessed the data on 25th January 2022)
2. Min and max vary wildly between datasets - going from -99% up to +250%
3. Australia, Japan and US datasets have missing values - I will impute/backfill these data below. Singapore has no missing data points.

## Preprocessing the data

First let's preprocess the data from the Google datasets.

The granularity of the data can go up to sub-region level 2 - i.e. local government area within a state/province of a nation.
The original dataset also requires some pivoting to make the columns (e.g. retail, public transport) into rows.
It is also divided by calendar year, so we'll need to handle that.

In [18]:
def preprocess(df):

    df = df[[
        'place_id', 
        'sub_region_1',
        'sub_region_2', 
        'date', 
        'retail_and_recreation_percent_change_from_baseline',
        'grocery_and_pharmacy_percent_change_from_baseline',
        'parks_percent_change_from_baseline',
        'transit_stations_percent_change_from_baseline',
        'workplaces_percent_change_from_baseline',
        'residential_percent_change_from_baseline'
        ]]
    
    # Impute/backfill - i.e. impute via last valid value
    target_columns = [ 
        'retail_and_recreation_percent_change_from_baseline',
        'grocery_and_pharmacy_percent_change_from_baseline',
        'parks_percent_change_from_baseline',
        'transit_stations_percent_change_from_baseline',
        'workplaces_percent_change_from_baseline',
        'residential_percent_change_from_baseline'
    ]
    df[target_columns] = df[target_columns].fillna(method='bfill')
    
    # Stack and clean-up column values
    df = df.rename(columns={
        'retail_and_recreation_percent_change_from_baseline': 'retail_and_recreation',
        'parks_percent_change_from_baseline': 'parks',
        'grocery_and_pharmacy_percent_change_from_baseline': 'grocery_and_pharmacy',
        'transit_stations_percent_change_from_baseline': 'public_transport',
        'workplaces_percent_change_from_baseline': 'workplaces',
        'residential_percent_change_from_baseline': 'residential'
    })
    df = df.set_index(['place_id', 'sub_region_1', 'sub_region_2', 'date'])
    df = df.stack(0)
    df = df.reset_index()
    df = df.rename(columns={
        0: "pct_change",
        'level_4': 'category',
    })

    return df

df_australia = preprocess(df_australia)
df_singapore = preprocess(df_singapore)
df_japan = preprocess(df_japan)
df_us = preprocess(df_us)

## Plotting the data

Next we will plot the data - one plot for each city.

I will create two subplots for each city -
1. Retail
2. Office vs Home
3. Parks and outdoor activities

Essentially these graphs will show how much traffic and how long people stayed at these particular destinations during a particular period. That is, if the category is above the baseline, it means they spent more time there than pre-pandemic.

In [19]:
# Plot settings
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
plt.style.use('fivethirtyeight')
sns.set_theme(style="whitegrid")
import matplotlib.ticker as mtick
%matplotlib inline

from pylab import rcParams
rcParams['figure.figsize'] = 11, 9 #Changes default matplotlib plots to this size

# DPI settings
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300

In [20]:
def plot_mobility_data(datasets):
    """Plots Mobility
    :param datasets:
    :param List[Dict]: 
    :rtype fig
    """
    # Format axis
    date_range = [d.strftime('%Y-%m-%d') for d in pd.date_range(start=data['date'].min(),end=data['date'].max(), freq="M")]

    # Identify annotations

    # Plot the charts
    fig, ax = plt.subplots(len(datasets),1, figsize=(25,10))

    for i in range(len(datasets)):
        plot_data = datasets[i]['dataset']

        sns.lineplot(
            ax=ax[i],
            data=plot_data,
            x='date',
            y='pct_change',
            hue='category',
            palette=datasets[i]['palette']
        )

        # Format the plot
        ax[i].yaxis.set_major_formatter(mtick.PercentFormatter())
        ax[i].legend(title = "Category", loc='best')
        ax[i].set_title(datasets[i]['title'], fontsize=20)
        ax[i].set_xticks(date_range)
        ax[i].set_xticklabels(labels=date_range, rotation=30)
        ax[i].set_ylabel('')  
        ax[i].set_xlabel('')

        # Annotations
        # https://www.who.int/en/activities/tracking-SARS-CoV-2-variants/
        # 1st Wave
        ax[i].axvline(
            x='2020-02-15',
            color='k',
            linestyle='--'   
        )
        ax[i].text(
            x='2020-02-15', 
            y=plot_data['pct_change'].max(), 
            s='Original Variant', 
            fontsize=12
        )

        # 2nd Wave - Alpha and Beta
        ax[i].axvline(
            x='2020-12-18',
            color='k',
            linestyle='--'   
        )
        ax[i].text(
            x='2020-12-18', 
            y=plot_data['pct_change'].max(), 
            s='Alpha/Beta Variant', 
            fontsize=12
        )

        # 3rd Wave - Delta
        ax[i].axvline(
            x='2021-04-04',
            color='k',
            linestyle='--'   
        )

        ax[i].text(
            x='2021-04-04', 
            y=plot_data['pct_change'].max(), 
            s='Delta Variant',  
            fontsize=12
        )

        # 4th Wave - Omicron
        ax[i].axvline(
            x='2021-11-24',
            color='k',
            linestyle='--'   
        )

        ax[i].text(
            x='2021-11-24', 
            y=plot_data['pct_change'].max(), 
            s='Omicron Variant',  
            fontsize=12
        )

    return fig, ax

In [21]:
"""
First plot Greater Brisbane area
"""

# Filter down to Greater Brisbane area
# https://brisbanetour.com.au/blog/what-is-the-greater-brisbane-area/
brisbane_LGAs = [
    'Brisbane City',
    'City of Ipswich',
    'Logan City',
    'Redland City',
    'Moreton Bay Region'
]
data = df_australia[df_australia['sub_region_2'].isin(brisbane_LGAs)]
# Aggregate these sub-regions by taking average
data = data.groupby(
    by=['date', 'category'],
    as_index=False
).mean()

datasets = [
    {
        'dataset': data[data['category'].isin(['retail_and_recreation', 'grocery_and_pharmacy'])],
        'title': 'Brisbane 2020 to 2022 (YTD) - Retail',
        'subtitle': 'Showing the impact of lockdowns on retail',
        'palette': 'crest'
    },
    {
        'dataset': data[data['category'].isin(['workplaces', 'residential', 'public_transport'])],
        'title': 'Brisbane 2020 to 2022 (YTD) - WFH, Office and Public Transport',
        'subtitle': 'Showing the impact of WFH and hybrid arrangements',
        'palette': 'magma'
    },
    {
        'dataset': data[data['category'].isin(['parks'])],
        'title': 'Brisbane 2020 to 2022 (YTD) - Parks',
        'subtitle': 'Showing the impact of outdoor activities',
        'palette': 'ocean'
    }
]

fig, ax = plot_mobility_data(datasets)

plt.tight_layout(pad=2.0)
plt.show()

## Observations for Brisbane:

1. Australia's 1st lockdown occurred roughly between March and June 2020. You can see how it affected workplace traffic.
2. Between April and July 2021, a series of snap lockdowns occurred. You can see the effects of the lockdown with a spike reduction in retail.
3. You can see the initial hit from the variants hit retail particularly hard.
4. Workplaces also show a significant drop in traffic right after a new variant, but gradually come back.
5. You can also see that workplace traffic has still not returned to pre-COVID levels.
6. While no lockdown has occurred in Brisbane since Omicron was discovered, there has been a de facto lockdown, with traffic dropping to lockdown levels.
7. Because Brisbane enjoyed a relatively lockdown-free state, the human traffic in parks didn't significantly go up.

In [22]:
"""
Next plot Sydney
"""

# Filter down to Greater Sydney area
# https://www.planning.nsw.gov.au/plans-for-your-area/a-metropolis-of-three-cities/greater-sydney-districts
sydney_LGAs = [
    "Blue Mountains City Council",
    "City of Blacktown",
    "Bayside Council",
    "Hornsby Shire",
    "Hawkesbury City Council",
    "Cumberland City Council",
    "Municipality of Burwood",
    "Municipality of Hunter's Hill",
    "Canterbury City Council",
    "Bankstown City Council",
    "City of Penrith",
    "City of Parramatta Council",
    "City of Canada Bay Council",
    "Ku–ring–gai Council",
    "Sutherland Shire Council",
    "Camden Council",
    "The Hills Shire",
    "Inner West Council",
    "Municipality of Lane Cove",
    "City of Campbelltown",
    "City of Randwick",
    "Northern Beaches Council",
    "City of Fairfield",
    "Municipality of Strathfield",
    "Mosman Municipal Council",
    "City of Liverpool",
    "Woollahra Municipality",
    "City of Willoughby",
    "Wollondilly Shire Council",
    "Waverley Council",
    "City of Ryde",
    "City of Sydney",
    "North Sydney Council"
]
data = df_australia[df_australia['sub_region_2'].isin(sydney_LGAs)]
# Aggregate these sub-regions by taking average
data = data.groupby(
    by=['date', 'category'],
    as_index=False
).mean()

datasets = [
    {
        'dataset': data[data['category'].isin(['retail_and_recreation', 'grocery_and_pharmacy'])],
        'title': 'Sydney 2020 to 2022 (YTD) - Retail',
        'subtitle': 'Showing the impact of lockdowns on retail',
        'palette': 'crest'
    },
    {
        'dataset': data[data['category'].isin(['workplaces', 'residential', 'public_transport'])],
        'title': 'Sydney 2020 to 2022 (YTD) - WFH, Office and Public Transport',
        'subtitle': 'Showing the impact of WFH and hybrid arrangements',
        'palette': 'magma'
    },
    {
        'dataset': data[data['category'].isin(['parks'])],
        'title': 'Sydney 2020 to 2022 (YTD) - Parks',
        'subtitle': 'Showing the impact of outdoor activities',
        'palette': 'ocean'
    }
]

fig, ax = plot_mobility_data(datasets)

plt.tight_layout(pad=2.0)
plt.show()

In [23]:
"""
Next plot Melbourne
"""

# Filter down to Greater Melbourne area
# https://www.sro.vic.gov.au/greater-melbourne-map-and-urban-zones
melbourne_LGAs = [
    "City of Banyule",
    "City of Bayside",
    "City of Boroondara",
    "City of Darebin",
    "City of Glen Eira​​",
    "City of Maribyrnong",
    "City of Monash",
    "City of Melbourne",
    "City of Moonee Valley",
    "City of Moreland",
    "Port Phillip City",
    "City of Stonnington",
    "Whitehorse City",
    "City of Yarra"
]
data = df_australia[df_australia['sub_region_2'].isin(melbourne_LGAs)]
# Aggregate these sub-regions by taking average
data = data.groupby(
    by=['date', 'category'],
    as_index=False
).mean()
datasets = [
    {
        'dataset': data[data['category'].isin(['retail_and_recreation', 'grocery_and_pharmacy'])],
        'title': 'Melbourne 2020 to 2022 (YTD) - Retail',
        'subtitle': 'Showing the impact of lockdowns on retail',
        'palette': 'crest'
    },
    {
        'dataset': data[data['category'].isin(['workplaces', 'residential', 'public_transport'])],
        'title': 'Melbourne 2020 to 2022 (YTD) - WFH, Office and Public Transport',
        'subtitle': 'Showing the impact of WFH and hybrid arrangements',
        'palette': 'magma'
    },
    {
        'dataset': data[data['category'].isin(['parks'])],
        'title': 'Melbourne 2020 to 2022 (YTD) - Parks',
        'subtitle': 'Showing the impact of outdoor activities',
        'palette': 'ocean'
    }
]

fig, ax = plot_mobility_data(datasets)

plt.tight_layout(pad=2.0)
plt.show()

## Observations for Sydney and Melbourne:

1. Melbourne had a 2nd major lockdown from around June 2020 to October 2020. You can see retail traffic drop in these periods.
2. Melbourne had a 3rd major lockdown from around June 2021 to October 2021. Again, retail traffic drops.
3. Sydney similiarly had a major lockdown from around June 2021 to October 2021.
3. Unlike Brisbane though, due to Melbourne going being the city with the longest number of lockdown days in the world, the Omicron variant has not resulted in a de facto lockdown. Besides the initial hit, traffic is still relatively stable.
4. Traffic to parks, however, actually still dropped even during lockdowns.

In [24]:
"""
Next plot Singapore
"""

data = df_singapore
datasets = [
    {
        'dataset': data[data['category'].isin(['retail_and_recreation', 'grocery_and_pharmacy'])],
        'title': 'Singapore 2020 to 2022 (YTD) - Retail',
        'subtitle': 'Showing the impact of lockdowns on retail',
        'palette': 'crest'
    },
    {
        'dataset': data[data['category'].isin(['workplaces', 'residential', 'public_transport'])],
        'title': 'Singapore 2020 to 2022 (YTD) - WFH, Office and Public Transport',
        'subtitle': 'Showing the impact of WFH and hybrid arrangements',
        'palette': 'magma'
    },
    {
        'dataset': data[data['category'].isin(['parks'])],
        'title': 'Singapore 2020 to 2022 (YTD) - Parks',
        'subtitle': 'Showing the impact of outdoor activities',
        'palette': 'ocean'
    }
]

fig = plot_mobility_data(datasets)

plt.tight_layout(pad=2.0)
plt.show()

## Observations for Singapore
1. Besides the initial lockdown, Singapore traffic for offices has maintained roughly the same, almost to pre-pandemic levels.
2. Park traffic hasn't significantly changed after the initial lockdown
3. Pharmacy traffic significantly increased to even above pre-pandemic levels

In [25]:
"""
Next plot Tokyo
"""

# Filter to Greater Tokyo area
# https://www.metro.tokyo.lg.jp/ENGLISH/ABOUT/HISTORY/history02.htm
tokyo_prefectures = [
    "Tokyo",
    "Saitama",
    "Chiba",
    "Kanagawa",
    "Ibaraki",
    "Tochigi",
    "Gunma",
    "Yamanashi"
]
data = df_japan[df_japan['sub_region_1'].isin(tokyo_prefectures)]
# Aggregate these sub-regions by taking average
data = data.groupby(
    by=['date', 'category'],
    as_index=False
).mean()
datasets = [
    {
        'dataset': data[data['category'].isin(['retail_and_recreation', 'grocery_and_pharmacy'])],
        'title': 'Tokyo 2020 to 2022 (YTD) - Retail',
        'subtitle': 'Showing the impact of lockdowns on retail',
        'palette': 'crest'
    },
    {
        'dataset': data[data['category'].isin(['workplaces', 'residential', 'public_transport'])],
        'title': 'Tokyo 2020 to 2022 (YTD) - WFH, Office and Public Transport',
        'subtitle': 'Showing the impact of WFH and hybrid arrangements',
        'palette': 'magma'
    },
    {
        'dataset': data[data['category'].isin(['parks'])],
        'title': 'Tokyo 2020 to 2022 (YTD) - Parks',
        'subtitle': 'Showing the impact of outdoor activities',
        'palette': 'ocean'
    }
]

fig = plot_mobility_data(datasets)

plt.tight_layout(pad=2.0)
plt.show()

## Observations for Tokyo
1. Despite the 2021 Summer Olympics, Tokyo traffic didn't significantly change for this period
2. Office traffic is almost at pre-pandemic levels, showing the contrast between cultural acceptance of WFH/hybrid arrangements vs other cities (e.g. Brisbane/Melbourne)
3. New variants caused an initial dip in traffic, but it quickly recovered after that

In [26]:
"""
Next plot New York City
"""
# Filter to NYC 5 boroughs
data = df_us[df_us['sub_region_2'].isin(['New York County', 'Kings County', 'Bronx County', 'Richmond County', 'Queens County'])]
# Aggregate these sub-regions by taking average
data = data.groupby(
    by=['date', 'category'],
    as_index=False
).mean()
datasets = [
    {
        'dataset': data[data['category'].isin(['retail_and_recreation', 'grocery_and_pharmacy'])],
        'title': 'New York City 2020 to 2022 (YTD) - Retail',
        'subtitle': 'Showing the impact of lockdowns on retail',
        'palette': 'crest'
    },
    {
        'dataset': data[data['category'].isin(['workplaces', 'residential', 'public_transport'])],
        'title': 'New York City 2020 to 2022 (YTD) - WFH, Office and Public Transport',
        'subtitle': 'Showing the impact of WFH and hybrid arrangements',
        'palette': 'magma'
    },
    {
        'dataset': data[data['category'].isin(['parks'])],
        'title': 'New York City 2020 to 2022 (YTD) - Parks',
        'subtitle': 'Showing the impact of outdoor activities',
        'palette': 'ocean'
    }
]

fig = plot_mobility_data(datasets)

plt.tight_layout(pad=2.0)
plt.show()

# Observations for New York
1. Retail plummeted during initial wave, but bounced back to almost pre-pandemic levels (until Omicron)
2. There has been a steady increase of park traffic since the pandemic to even above pre-pandemic levels
3. Emergence of new variants didn't significantly impact traffic, besides an initial dip

## Conclusions
So after analysing traffic for these cities, it is interesting how government policy and cultural attitidues have shaped human traffic.

It seems the largest factors contributing to dips in traffic are (obviously):
1. Lockdowns
2. New variants emerging